Initial Imports

In [1]:
import csv
import time

import openmeteo_requests
import requests_cache
from retry_requests import retry
import pandas as pd
from datetime import datetime, timedelta

# Generate map of cities and their respective coordinates

In [2]:
# Replace 'file.csv' with your CSV file's name and path
file_path = 'data/tr.csv'

# Create an empty dictionary to store city coordinates
city_coordinates = {}

# Open the CSV file and read data
with open(file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        city = row['city']
        lat = float(row['lat'])
        lng = float(row['lng'])
        city_coordinates[city] = (lat, lng)

# Display the dictionary containing city coordinates
for city, coordinates in city_coordinates.items():
    print(f"City: {city}, Coordinates: {coordinates}")
city_coordinates

City: Istanbul, Coordinates: (41.0136, 28.955)
City: Ankara, Coordinates: (39.93, 32.85)
City: İzmir, Coordinates: (38.42, 27.14)
City: Bursa, Coordinates: (40.1833, 29.05)
City: Antalya, Coordinates: (36.8874, 30.7075)
City: Konya, Coordinates: (37.8667, 32.4833)
City: Gaziantep, Coordinates: (37.0628, 37.3792)
City: Şanlıurfa, Coordinates: (37.1583, 38.7917)
City: Mersin, Coordinates: (36.8, 34.6333)
City: Adana, Coordinates: (37.0, 35.3213)
City: Diyarbakır, Coordinates: (37.91, 40.24)
City: Kayseri, Coordinates: (38.7225, 35.4875)
City: Samsun, Coordinates: (41.2903, 36.3336)
City: Akçaabat, Coordinates: (41.0167, 39.55)
City: Denizli, Coordinates: (37.7667, 29.0833)
City: Çankaya, Coordinates: (39.9244, 32.8856)
City: Esenyurt, Coordinates: (41.047, 28.658)
City: Eskişehir, Coordinates: (39.7767, 30.5206)
City: Seyhan, Coordinates: (36.9831, 35.3328)
City: Küçükçekmece, Coordinates: (41.0, 28.8)
City: Erzurum, Coordinates: (39.9086, 41.2769)
City: Bağcılar, Coordinates: (41.0344, 

{'Istanbul': (41.0136, 28.955),
 'Ankara': (39.93, 32.85),
 'İzmir': (38.42, 27.14),
 'Bursa': (40.1833, 29.05),
 'Antalya': (36.8874, 30.7075),
 'Konya': (37.8667, 32.4833),
 'Gaziantep': (37.0628, 37.3792),
 'Şanlıurfa': (37.1583, 38.7917),
 'Mersin': (36.8, 34.6333),
 'Adana': (37.0, 35.3213),
 'Diyarbakır': (37.91, 40.24),
 'Kayseri': (38.7225, 35.4875),
 'Samsun': (41.2903, 36.3336),
 'Akçaabat': (41.0167, 39.55),
 'Denizli': (37.7667, 29.0833),
 'Çankaya': (39.9244, 32.8856),
 'Esenyurt': (41.047, 28.658),
 'Eskişehir': (39.7767, 30.5206),
 'Seyhan': (36.9831, 35.3328),
 'Küçükçekmece': (41.0, 28.8),
 'Erzurum': (39.9086, 41.2769),
 'Bağcılar': (41.0344, 28.8564),
 'Pendik': (40.8747, 29.235),
 'Selçuklu': (37.8814, 32.4922),
 'Yıldırım': (40.11, 29.0821),
 'Kartal': (40.8872, 29.19),
 'Batman': (37.887, 41.132),
 'Esenler': (41.0339, 28.8903),
 'Kahramanmaraş': (37.5833, 36.9333),
 'Kâğıthane': (41.0719, 28.9664),
 'Avcılar': (40.9792, 28.7214),
 'Trabzon': (41.005, 39.7225),
 '

In [3]:

today = datetime.now()  # Get today's date
one_year_ago = today - timedelta(days=365)  # Subtract 365 days for one year
today

datetime.datetime(2024, 1, 6, 19, 31, 25, 249540)

In [4]:
def format_date(date):
	formatted_date = date.strftime('%Y-%m-%d')
	return formatted_date

In [5]:
class OpenMeteoRequestsError(Exception):
    pass

In [6]:
def api_call(lat,long):
	# Setup the Open-Meteo API client with cache and retry on error
	cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
	retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
	openmeteo = openmeteo_requests.Client(session = retry_session)
	
	# Make sure all required weather variables are listed here
	# The order of variables in hourly or daily is important to assign them correctly below
	url = "https://archive-api.open-meteo.com/v1/archive"
	params = {
		"latitude": lat,
		"longitude": long,
		"start_date": format_date(one_year_ago),
		"end_date": format_date(today),
		"hourly": ["temperature_2m", "relative_humidity_2m", "rain", "snowfall", "surface_pressure", "cloud_cover", "wind_speed_10m"]
	}
	responses = openmeteo.weather_api(url, params=params)
	
	# Process first location. Add a for-loop for multiple locations or weather models
	response = responses[0]
	print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
	print(f"Elevation {response.Elevation()} m asl")
	print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
	print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")
	
	# Process hourly data. The order of variables needs to be the same as requested.
	hourly = response.Hourly()
	hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
	hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
	hourly_rain = hourly.Variables(2).ValuesAsNumpy()
	hourly_snowfall = hourly.Variables(3).ValuesAsNumpy()
	hourly_surface_pressure = hourly.Variables(4).ValuesAsNumpy()
	hourly_cloud_cover = hourly.Variables(5).ValuesAsNumpy()
	hourly_wind_speed_10m = hourly.Variables(6).ValuesAsNumpy()
	
	hourly_data = {"date": pd.date_range(
		start=pd.to_datetime(hourly.Time(), unit="s"),
		end=pd.to_datetime(hourly.TimeEnd(), unit="s"),
		freq=pd.Timedelta(seconds=hourly.Interval()),
		inclusive="left"
	), "temperature_2m": hourly_temperature_2m, "relative_humidity_2m": hourly_relative_humidity_2m, "rain": hourly_rain,
		"snowfall": hourly_snowfall, "surface_pressure": hourly_surface_pressure, "cloud_cover": hourly_cloud_cover,
		"wind_speed_10m": hourly_wind_speed_10m}
	
	hourly_dataframe = pd.DataFrame(data = hourly_data)
	return hourly_dataframe

In [7]:
data_frame = pd.DataFrame()

for i, (city, (lat, lng)) in enumerate(city_coordinates.items(), start=1):
    df = api_call(lat, lng)

    if data_frame.empty:
        data_frame = df
    else:
        data_frame = pd.concat((data_frame, df))

    if i % 100 == 0:
        print(f"Reached iteration {i}, waiting for 65 seconds before continuing...")
        time.sleep(65)  # Wait for 65 seconds
data_frame

Coordinates 41.01932907104492°E 29.01569175720215°N
Elevation 41.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 39.964847564697266°E 32.68156433105469°N
Elevation 848.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 38.418277740478516°E 27.19512176513672°N
Elevation 13.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 40.17574691772461°E 28.90603256225586°N
Elevation 257.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 36.8717041015625°E 30.789474487304688°N
Elevation 46.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 37.85588455200195°E 32.573726654052734°N
Elevation 1025.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 37.08259963989258°E 37.450462341308594°N
Elevation 842.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 37.152896881103516°E 38.80952453613281°N
Elevation 525.0 m asl
Timezone None None
Timezone difference to

,date,temperature_2m,relative_humidity_2m,rain,snowfall,surface_pressure,cloud_cover,wind_speed_10m
0,2023-01-06 00:00:00,9.0735,89.105789,0.0,0.0,1013.260986,29.400002,18.356470
1,2023-01-06 01:00:00,9.1235,88.503242,0.0,0.0,1013.460999,24.300001,18.598450
2,2023-01-06 02:00:00,9.0735,87.895729,0.0,0.0,1013.360535,25.200001,18.335276
3,2023-01-06 03:00:00,9.1235,86.705132,0.0,0.0,1013.460999,100.000000,18.864824
4,2023-01-06 04:00:00,9.4235,86.438721,0.1,0.0,1013.565857,93.900002,17.819090
